In [1]:
##Estimación de indicadores de mujeres
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,lubridate, showtext,srvyr,foreign)
font_add_google("Poppins", "pop")
showtext_auto()

Loading required package: pacman



In [2]:
##Víctimas en carpetas de investigación
victimas<-read_csv('https://archivo.datos.cdmx.gob.mx/FGJ/victimas/victimasFGJ_acumulado_2024_09.csv')%>%
janitor::clean_names()%>%
#Filtrar mujeres
filter(sexo == 'Femenino',
       tipo_persona=='FISICA',
       alcaldia_hecho != 'FUERA DE CDMX')%>%
#Clasificar delitos con perspectiva de género
mutate(
    perspectiva_genero = case_when(delito %in% c('ABUSO SEXUAL',
                                                 'ACOSO SEXUAL',
                                                 'ACOSO SEXUAL AGRAVADO EN CONTRA DE MENORES',
                                                 'CONTRA LA INTIMIDAD SEXUAL',
                                                 'FEMINICIDIO',
                                                 'FEMINICIDIO POR ARMA BLANCA',
                                                 'FEMINICIDIO POR DISPARO DE ARMA DE FUEGO',
                                                 'FEMINICIDIO POR GOLPES',
                                                 'HOMICIDIO POR ARMA BLANCA',
                                                 'HOMICIDIO POR ARMA DE FUEGO',
                                                 'HOMICIDIO POR GOLPES',
                                                 'HOMICIDIOS INTENCIONALES (OTROS)',
                                                 'LESIONES INTENCIONALES',
                                                 'LESIONES INTENCIONALES POR ARMA BLANCA',
                                                 'LESIONES INTENCIONALES POR ARMA DE FUEGO',
                                                  'LESIONES INTENCIONALES POR GOLPES',
                                                 'PLAGIO O SECUESTRO',
                                                 'TRATA DE PERSONAS',
                                                 'VIOLACION',
                                                 'VIOLACION EQUIPARADA',
                                                 'VIOLACION EQUIPARADA POR CONOCIDO',
                                                 'VIOLACION TUMULTUARIA',
                                                 'VIOLENCIA FAMILIAR') ~ 1,
                            TRUE ~ 0))%>%
#Dejar solo aquellos que han sido víctimas
  filter(calidad_juridica %in% c(
    'OFENDIDO',
    'VICTIMA Y DENUNCIANTE',
    'VICTIMA',
    'OFENDIDO Y DENUNCIANTE',
    'CADAVER',
    'AGRAVIADO',
    'LESIONADO',
    'DENUNCIANTE Y VICTIMA'
  ))%>%
#Clasificar por delito general
mutate(
    delito_general = case_when(delito %in% c('ABUSO SEXUAL') ~ "ABUSO SEXUAL",
                                delito %in% c('ACOSO SEXUAL', 'ACOSO SEXUAL AGRAVADO EN CONTRA DE MENORES') ~ "ACOSO SEXUAL",
                                delito %in% c('CONTRA LA INTIMIDAD SEXUAL') ~ "CONTRA LA INTIMIDAD SEXUAL",
                                delito %in% c('FEMINICIDIO', 'FEMINICIDIO POR ARMA BLANCA', 'FEMINICIDIO POR DISPARO DE ARMA DE FUEGO', 'FEMINICIDIO POR GOLPES') ~ "FEMINICIDIO",
                                delito %in% c('HOMICIDIO POR ARMA BLANCA', 'HOMICIDIO POR ARMA DE FUEGO', 'HOMICIDIO POR GOLPES', 'HOMICIDIOS INTENCIONALES (OTROS)') ~ "HOMICIDIO DOLOSO",
                                delito %in% c('LESIONES INTENCIONALES', 'LESIONES INTENCIONALES POR ARMA BLANCA', 'LESIONES INTENCIONALES POR ARMA DE FUEGO', 'LESIONES INTENCIONALES POR GOLPES') ~ "LESIONES DOLOSAS",
                                delito %in% c('PLAGIO O SECUESTRO') ~ "SECUESTRO",
                                delito %in% c('TRATA DE PERSONAS') ~ "TRATA DE PERSONAS",
                                delito %in% c('VIOLACION', 'VIOLACION EQUIPARADA', 'VIOLACION EQUIPARADA POR CONOCIDO', 'VIOLACION TUMULTUARIA') ~ "VIOLACION",
                                delito %in% c('VIOLENCIA FAMILIAR') ~ "VIOLENCIA FAMILIAR",
                                TRUE ~ "OTROS"
    ))



Rows: 1415763 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): mes_inicio, mes_hecho, delito, categoria_delito, sexo, tipo_perso...
dbl   (5): anio_inicio, anio_hecho, edad, latitud, longitud
date  (2): fecha_inicio, fecha_hecho
time  (2): hora_inicio, hora_hecho

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
#Víctimas anual

victimas%>%
filter(perspectiva_genero==1, anio_inicio>2018)%>%
group_by(anio_inicio)%>%
summarise(vict=n())%>%
#Calcular promedio diario de acuerdo al año
mutate(promedio_diario=vict/365)

anio_inicio,vict,promedio_diario
<dbl>,<int>,<dbl>
2019,28658,78.51507
2020,26082,71.45753
2021,37095,101.63014
2022,40431,110.76986
2023,40775,111.71233
2024,29962,82.08767


In [4]:
#Víctimas de feminicidio
victimas%>%
filter(delito_general=='FEMINICIDIO', anio_inicio>2018)%>%
group_by(anio_inicio)%>%
summarise(vict=n())%>%
#Calcular promedio diario de acuerdo al año
mutate(promedio_diario=vict/365)

anio_inicio,vict,promedio_diario
<dbl>,<int>,<dbl>
2019,70,0.1917808
2020,70,0.1917808
2021,61,0.1671233
2022,78,0.2136986
2023,63,0.1726027
2024,43,0.1178082


### Seguridad y violencia

### Episodios de violencia contra las mujeres

In [5]:
urls<-c("https://www.inegi.org.mx/contenidos/programas/endireh/2021/microdatos/bd_endireh_2021_csv.zip",
        "https://www.inegi.org.mx/contenidos/programas/endireh/2016/microdatos/bd_mujeres_endireh2016_sitioinegi_csv.zip")

In [6]:
#Crear carpeta "endireh" en "datos"
if (!dir.exists("datos/endireh")) {
  dir.create("datos/endireh")
}

In [7]:
#Descargar, descomprimir y borrar zip en la carpeta "datos"
options(timeout = 900)
for (i in 1:length(urls)){
  temp <- tempfile()
  download.file(urls[i], temp, mode = "wb")
  unzip(temp, exdir = "datos/endireh")
  unlink(temp)
}


In [8]:
estima_violencia <- function(year) {
  # Seleccionar archivo base según el año
  archivo <- switch(as.character(year),
                    "2021" = "bd_endireh_2021.csv",
                    "2016" = "bd_mujeres_endireh2016_sitioinegi.csv",
                    stop("Año no válido. Prueba con 2021 o 2016.")
  )

  # Definir ruta base según el año
  ruta_base <- ifelse(year == 2021,
                      "datos/endireh/bd_endireh_2021_csv/",
                      "datos/endireh/bd_mujeres_endireh2016_sitioinegi_csv/")

  # Cargar tabla principal
  tabla_violencia <- read_csv(paste0(ruta_base, "TB_SEC_IV.csv")) %>%
    janitor::clean_names()

  # Definir secciones a unir
  romanos <- c("V", "VI", "VII", "VIII", "IX", "X", "XI", "XII",
               "XIII", "XIV", "XV", "XVI")

  # Incluir XIV_2 solo si el año es 2021
  if (year == 2021) {
    romanos <- c(romanos, "XIV_2")
  }

  # Unir todas las secciones
  for (i in romanos) {
    tabla <- read_csv(paste0(ruta_base, "TB_SEC_", i, ".csv")) %>%
      janitor::clean_names()

    tabla_violencia <- left_join(tabla_violencia, tabla,
                                 c("upm","estrato","upm_dis","est_dis","fac_muj", "viv_sel", "hogar","fac_viv","id_viv","cve_ent","nom_ent"))
  }


  #Clasificar violencia en los últimos 12 meses
tabla_violencia <- tabla_violencia %>%
  mutate(
    violencia_muj = case_when(
      p7_8_1 %in% c('1', '2', '3') | p7_8_2 %in% c('1', '2', '3') |
      p7_8_3 %in% c('1', '2', '3') | p7_8_4 %in% c('1', '2', '3') |
      p7_8_5 %in% c('1', '2', '3') | p7_8_6 %in% c('1', '2', '3') |
      p7_8_7 %in% c('1', '2', '3') | p7_8_8 %in% c('1', '2', '3') |
      p7_8_9 %in% c('1', '2', '3') | p7_8_10 %in% c('1', '2', '3') |
      p7_8_11 %in% c('1', '2', '3') | p7_8_12 %in% c('1', '2', '3') |
      p7_8_13 %in% c('1', '2', '3') | p7_8_14 %in% c('1', '2', '3') |
      p7_8_15 %in% c('1', '2', '3') | p7_8_16 %in% c('1', '2', '3') |
      p7_8_17 %in% c('1', '2', '3') | p7_8_18 %in% c('1', '2', '3') |
      p8_8_1 %in% c('1') | p8_8_2 %in% c('1') |
        p8_8_3 %in% c('1') | p8_8_4 %in% c('1') |
        p8_8_5 %in% c('1') | p8_8_6 %in% c('1') |
        p8_8_7 %in% c('1') | p8_8_8 %in% c('1') |
        p8_8_9 %in% c('1') |
        p8_11_1 %in% c('1', '2', '3') | p8_11_2 %in% c('1', '2', '3') |
        p8_11_3 %in% c('1', '2', '3') | p8_11_4 %in% c('1', '2', '3') |
        p8_11_5 %in% c('1', '2', '3') | p8_11_6 %in% c('1', '2', '3') |
        p8_11_7 %in% c('1', '2', '3') | p8_11_8 %in% c('1', '2', '3') |
      p8_11_9 %in% c('1', '2', '3') | p8_11_10 %in% c('1', '2', '3') |
      p8_11_11 %in% c('1', '2', '3') | p8_11_12 %in% c('1', '2', '3') |
      p8_11_13 %in% c('1', '2', '3') | p8_11_14 %in% c('1', '2', '3') |
      p8_11_15 %in% c('1', '2', '3') | p8_11_16 %in% c('1', '2', '3') |
      p8_11_17 %in% c('1', '2', '3') | p8_11_18 %in% c('1', '2', '3') |
      p8_11_19 %in% c('1', '2', '3') | p9_3_1 %in% c('1', '2', '3') |
      p9_3_2 %in% c('1', '2', '3') | p9_3_3 %in% c('1', '2', '3') |
      p9_3_4 %in% c('1', '2', '3') | p9_3_5 %in% c('1', '2', '3') |
      p9_3_6 %in% c('1', '2', '3') | p9_3_7 %in% c('1', '2', '3') |
      p9_3_8 %in% c('1', '2', '3') | p9_3_9 %in% c('1', '2', '3') |
      p9_3_10 %in% c('1', '2', '3') | p9_3_11 %in% c('1', '2', '3') |
      p9_3_12 %in% c('1', '2', '3') | p9_3_13 %in% c('1', '2', '3') |
      p9_3_14 %in% c('1', '2', '3') | p9_3_15 %in% c('1', '2', '3') |
      p9_3_16 %in% c('1', '2', '3') | p11_1_1 %in% c('1', '2', '3') |
      p11_1_2 %in% c('1', '2', '3') | p11_1_3 %in% c('1', '2', '3') |
      p11_1_4 %in% c('1', '2', '3') | p11_1_5 %in% c('1', '2', '3') |
      p11_1_6 %in% c('1', '2', '3') | p11_1_7 %in% c('1', '2', '3') |
      p11_1_8 %in% c('1', '2', '3') | p11_1_9 %in% c('1', '2', '3') |
      p11_1_10 %in% c('1', '2', '3') | p11_1_11 %in% c('1', '2', '3') |
      p11_1_12 %in% c('1', '2', '3') | p11_1_13 %in% c('1', '2', '3') |
      p11_1_14 %in% c('1', '2', '3') | p11_1_15 %in% c('1', '2', '3') |
      p11_1_16 %in% c('1', '2', '3') | p11_1_17 %in% c('1', '2', '3') |
      p11_1_18 %in% c('1', '2', '3') | p11_1_19 %in% c('1', '2', '3') |
      p11_1_20 %in% c('1', '2', '3') | p14_3_1 %in% c('1', '2', '3') |
      p14_3_2 %in% c('1', '2', '3') | p14_3_3 %in% c('1', '2', '3') |
      p14_3_4 %in% c('1', '2', '3') | p14_3_5 %in% c('1', '2', '3') |
      p14_3_6 %in% c('1', '2', '3') | p14_3_7 %in% c('1', '2', '3') |
      p14_3_8 %in% c('1', '2', '3') | p14_3_9 %in% c('1', '2', '3') |
      p14_3_10 %in% c('1', '2', '3') | p14_3_11 %in% c('1', '2', '3') |
      p14_3_12 %in% c('1', '2', '3') | p14_3_13 %in% c('1', '2', '3') |
      p14_3_14 %in% c('1', '2', '3') | p14_3_15 %in% c('1', '2', '3') |
      p14_3_16 %in% c('1', '2', '3') | p14_3_17 %in% c('1', '2', '3') |
      p14_3_18 %in% c('1', '2', '3') | p14_3_19 %in% c('1', '2', '3') |
      p14_3_20 %in% c('1', '2', '3') | p14_3_21 %in% c('1', '2', '3') |
      p14_3_22 %in% c('1', '2', '3') | p14_3_23ab %in% c('1', '2', '3') |
      p14_3_24ab %in% c('1', '2', '3') | p14_3_25 %in% c('1', '2', '3') |
      p14_3_26 %in% c('1', '2', '3') | p14_3_27 %in% c('1', '2', '3') |
        p14_3_28 %in% c('1', '2', '3') | p14_3_29 %in% c('1', '2', '3') |
        p14_3_30 %in% c('1', '2', '3') | p14_3_31 %in% c('1', '2', '3') |
        p14_3_32 %in% c('1', '2', '3') | p14_3_33 %in% c('1', '2', '3') |
        p14_3_34 %in% c('1', '2', '3') | p14_3_35ab %in% c('1', '2', '3') |
        p14_3_36ab %in% c('1', '2', '3') | p14_3_37ab %in% c('1', '2', '3') |
        p14_3_38ab %in% c('1', '2', '3') ~ 1,
      TRUE ~ 0
    )
  )

dm<-tabla_violencia%>%
  as_survey_design(
  weights = fac_muj,
    ids = upm_dis,
    strata = est_dis,
    nest = TRUE
  )
dm %>%
  filter(cve_ent == "09") %>%
group_by(violencia_muj) %>%
summarise(
  pob = survey_total(vartype = "cv"),
  porcentaje = survey_prop(vartype = "cv")
) %>% mutate(porcentaje = porcentaje * 100)%>%
filter(violencia_muj == 1)

}

In [9]:
estima_violencia(2021)

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 110127 Columns: 75
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (34): ID_VIV, ID_PER, UPM, VIV_SEL, N_REN, DOMINIO, CVE_ENT, NOM_ENT, CV...
dbl (39): HOGAR, P4AB_1, P4A_1, P4A_2, P4B_1, P4B_2, P4BC_1, P4C_1, P4BC_3, ...
lgl  (2): P4_10_2_3, P4_10_3_3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 110127 Columns: 39
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (15): ID_VIV, ID_PER, UPM, VIV_SEL, N_REN, DOMINIO, CVE_ENT, NOM_ENT, CV...
dbl (20): HOGAR, P5_1, P5_3_1, P5_3_2, P5_3_3, P5_3_4,

violencia_muj,pob,pob_cv,porcentaje,porcentaje_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1880387,0.0284019,46.13754,0.02465223


In [10]:
##Leer archivo de excel para obtener resultados rápidos
url_excel<-"https://www.inegi.org.mx/contenidos/programas/endireh/2021/tabulados/xxi_prevalencia_de_la_violencia_estimaciones_endireh2021.xlsx"
violencia<-openxlsx::read.xlsx(url_excel,sheet=3,
rows=62:64,cols=1:10)

In [11]:
violencia_final<-violencia%>%
#seleccionar columnas 2 y 7
select(c(2,7))%>%rename(violencia_2021=1,
                        violencia_2016=2)
violencia_final

,violencia_2021,violencia_2016
,<dbl>,<dbl>
1,46.13754,55.06094


### Percepción de inseguridad

In [12]:
dir.create('datos/envipe', showWarnings = FALSE)
url_1 <- 'https://www.inegi.org.mx/contenidos/programas/envipe/'
url_2 <- '/microdatos/bd_envipe_'
years <- c(2020:2024)
data_list <- list()

In [13]:
# Ciclo 1: Descarga y descompresión
for (i in 1:length(years)) {
    if (years[i] %in% c(2024)) {
        fin <- '_csv.zip'
    } else {
        fin <- '_dbf.zip'
    }

    url <- paste0(url_1, years[i], url_2, years[i], fin)
    destfile <- paste0('datos/envipe/', years[i], fin)
    folder <- paste0('datos/envipe/', years[i])

    dir.create(folder, showWarnings = FALSE)

    tryCatch(
        {
            options(timeout = 9999999)
            download.file(url, destfile, mode = 'wb')
        },
        error = function(e) {
            message('Error en la descarga: ', e)
        }
    )

    unzip_result <- tryCatch(
        {
            unzip(destfile, exdir = folder)
            TRUE
        },
        error = function(e) {
            message('Error en la descompresión: ', e)
            FALSE
        }
    )

    if (unzip_result) {
        print(paste('Archivos descomprimidos para el año', years[i]))

        # Eliminar el archivo ZIP después de la descompresión
        tryCatch(
            {
                file.remove(destfile)
                print(paste('Archivo eliminado:', destfile))
            },
            error = function(e) {
                message('Error al eliminar el archivo:', e)
            }
        )
    }
}


[1] "Archivos descomprimidos para el año 2020"
[1] "Archivo eliminado: datos/envipe/2020_dbf.zip"
[1] "Archivos descomprimidos para el año 2021"
[1] "Archivo eliminado: datos/envipe/2021_dbf.zip"
[1] "Archivos descomprimidos para el año 2022"
[1] "Archivo eliminado: datos/envipe/2022_dbf.zip"
[1] "Archivos descomprimidos para el año 2023"
[1] "Archivo eliminado: datos/envipe/2023_dbf.zip"
[1] "Archivos descomprimidos para el año 2024"
[1] "Archivo eliminado: datos/envipe/2024_csv.zip"


In [200]:
estima_inseguridad <- function(year) {

# Definir rutas
if  (year %in% c(2020:2021)) {
    ruta_base <- paste0('datos/envipe/',year,'/bd_envipe_', year, '_dbf/')
}else {
    ruta_base <- paste0('datos/envipe/', year, '/')
}


#Leer datos
if (year %in% c(2020:2023)) {
    seguridad <- read.dbf(paste0(ruta_base, 'TPer_Vic1.dbf'))

} else {
    seguridad <- read_csv(paste0(ruta_base, 'TPer_Vic1.csv'))
}

seguridad<-seguridad%>%
    janitor::clean_names()%>%
    mutate(fac_ele=as.numeric(fac_ele), sexo=as.numeric(sexo))

#Crear diseño muestral
dm<-seguridad%>%
    as_survey_design(
    weights = fac_ele,
    ids = upm_dis,
    strata = est_dis,
    nest = TRUE
    )


  #Estimar percepción de inseguridad de mujeres en la Ciudad de México
    resultado<-dm %>%
    filter(cve_ent == "09") %>%
    filter(ap4_3_3!=3)%>%
  mutate(tipo_seg=case_when(
    ap4_3_3 %in% c(1) ~ "Seguro",
    ap4_3_3 %in% c(2) ~ "Inseguro",
    ap4_3_3 %in% c(5) ~ "No aplica",
    ap4_3_3 %in% c(9) ~ "No sabe"))%>%
  group_by(sexo,tipo_seg) %>%
  summarise(
    pob = survey_total(vartype = "cv"),
    porcentaje = survey_prop(vartype = "cv")
  ) %>% mutate(porcentaje = porcentaje * 100)%>%
  filter(tipo_seg == "Inseguro", sexo==2)%>%
    mutate(year = year)

    return(resultado)
}

In [201]:
tabla_inseg<-map_dfr(2020:2024,estima_inseguridad)
tabla_inseg

New names:
• `` -> `...1`
Rows: 89630 Columns: 232
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (21): ID_VIV, ID_HOG, ID_PER, UPM, VIV_SEL, HOGAR, RESUL_H, R_SEL, AREA...
dbl (211): ...1, SEXO, EDAD, AP4_1, AP4_2_01, AP4_2_02, AP4_2_03, AP4_2_04, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


sexo,tipo_seg,pob,pob_cv,porcentaje,porcentaje_cv,year
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2,Inseguro,3041434,0.01920830,86.74312,0.008427824,2020
2,Inseguro,3410235,0.01885379,86.63874,0.008147315,2021
2,Inseguro,3408583,0.02010401,85.74512,0.008338216,2022
2,Inseguro,3361871,0.01978396,83.76595,0.009211954,2023
2,Inseguro,3243254,0.02239651,78.70570,0.011078152,2024


### Empleo

In [16]:
# Función para descargar los datos de la ENOE en formato CSV
descargar_enoe_csv <- function() {
  # Crear directorio de trabajo si no existe
  if (!dir.exists("datos")) {
    dir.create("datos")
  }
  if (!dir.exists("datos/enoe")) {
    dir.create("datos/enoe")
  }

  # Cambiar al directorio de trabajo
  setwd("datos/enoe")

  # Función para descargar y extraer archivos
  descargar_extraer <- function(url) {
    temp <- tempfile()
    download.file(url, temp, mode = "wb")
    unzip(temp)
    unlink(temp)
  }

  # Descargar datos de 2005 a 2009
  for (i in 7:9) {
    url <- paste0("https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/200", i, "trim4_csv.zip")
    descargar_extraer(url)
  }

  # Descargar datos de 2010 a 2019
  for (i in 10:19) {
    url <- paste0("https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/20", i, "trim4_csv.zip")
    print(url)
    descargar_extraer(url)
  }

  # Descargar datos de 2020
  for (i in 20:20) {
    url <- paste0("https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_20", i, "_trim4_csv.zip")
    descargar_extraer(url)
  }

  # Descargar datos de 2021 a 2022
  for (i in 21:22) {
    url <- paste0("https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_20", i, "_trim4_csv.zip")
    descargar_extraer(url)
  }

  # Descargar datos de 2023
  for (i in 23:23) {
    url <- paste0("https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_20", i, "_trim4_csv.zip")
    descargar_extraer(url)
  }

  # Eliminar archivos innecesarios (conservar solo SDEMT y COE1T)
  # Listar archivos con extensión .csv, sin importar mayúsculas/minúsculas
  archivos <- list.files(pattern = "\\.csv$", ignore.case = TRUE)

  # Lista de archivos a eliminar (que NO contienen "SDEMT" o "COE1T", sin importar mayúsculas/minúsculas)
  archivos_a_eliminar <- archivos[!grepl("sdemt|coe1t", archivos, ignore.case = TRUE)]
  file.remove(archivos_a_eliminar)

  # Eliminar archivo zip (si existe)
  if (file.exists("archivos.zip")) {
    file.remove("archivos.zip")
  }

  # Regresar al directorio original
  setwd("../..")
}

In [17]:
descargar_enoe_csv()

[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2010trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2011trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2012trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2013trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2014trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2015trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2016trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2017trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2018trim4_csv.zip"
[1] "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/2019trim4_csv.zip"


In [18]:
# Función para extraer el año desde el nombre del archivo
extraer_anio <- function(path) {
  anio_str <- str_extract(path, "\\d{2}(?=\\D*$)") # Extrae los últimos dos dígitos del año
  anio <- as.numeric(paste0("20", anio_str)) # Convierte a formato 20XX
  return(anio)
}

In [19]:
# Función para procesar la ENOE y estimar el empleo cultural
estima_empleo_cultura <- function(sdemt_path, coe1t_path) {
  year <- extraer_anio(sdemt_path)

  # Leer y filtrar ocupados
  ocupados <- read_csv(sdemt_path, show_col_types = FALSE) %>%
    janitor::clean_names() %>%
    filter(
      clase2 == 1,
      (eda >= 15 & eda <= 98 & !is.na(eda)) & (c_res == 1 | c_res == 3)
    ) %>%
    mutate(year = year) # Agregar el año

  # Leer COE1T y seleccionar columnas según el año
  sinco <- read_csv(coe1t_path, show_col_types = FALSE) %>%
    janitor::clean_names() %>%
    { if (year <= 2019) select(., cd_a, ent, con, v_sel, n_hog, h_mud, n_ren, p3) else
        select(., tipo, mes_cal, cd_a, ent, con, v_sel, n_hog, h_mud, n_ren, p3) } %>%
    rename(sinco = p3)

  # Hacer el `left_join()` sin `tipo` si el año está entre 2005 y 2019
  if (year <= 2019) {
    ocupados <- left_join(ocupados, sinco, by = c("cd_a", "ent", "con", "v_sel", "n_hog", "h_mud", "n_ren"))
  } else {
    ocupados <- left_join(ocupados, sinco, by = c("tipo", "mes_cal", "cd_a", "ent", "con", "v_sel", "n_hog", "h_mud", "n_ren"))
  }

  # Clasificación de empleo cultural
  ocupados <- ocupados %>%
    mutate(
             cultura_cdmx_st = ifelse(sinco %in% c(
        2175, 7611, 2552, 2131, 7313, 7342, 7321, 2174, 7343, 2173, 2171, 1721, 1722,
        2544, 2271, 2625, 2162, 1422, 1322, 1421, 2541, 2543, 2542, 2164, 2151, 2163, 2144, 2425, 2655, 3222, 7322, 2712,
        7223, 2551, 5212, 2636, 4233, 2172, 8155, 8144, 7999, 7344, 2553, 7352, 2152, 2161, 7323, 7201, 7601, 7401, 7301,
        2652, 7332, 9713, 4312, 7614, 7353, 2514, 5212, 2653, 2654, 7299, 7999, 7311, 7331, 7341, 7411, 7412, 9234, 9235,
        9237, 4233, 2141, 2153, 2339, 2334, 2715
      ), 1, 0)

    )

  # Crear diseño de encuesta
  options(survey.lonely.psu = "certainty")
  if (year <= 2019) {
    dm <- ocupados %>%
      as_survey_design(
        ids = upm,
        strata = est_d,
        weights = fac,
        nest = TRUE
      )
  } else {
    dm <- ocupados %>%
      as_survey_design(
        ids = upm,
        strata = est_d_tri,
        weights = fac_tri,
        nest = TRUE
      )
  }



  resultado<-dm %>%
      filter(ent == 9) %>%
      group_by(cultura_cdmx_st,sex) %>%
      summarise(
        ocupados = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
      ) %>%
      mutate(porcentaje = porcentaje * 100) %>%
      filter(cultura_cdmx_st == 1,sex==2)%>%
              mutate(year = year)%>%
      filter(year>=2012)



  return(resultado)
}

In [20]:
lista_archivos <- list.files(path = "datos/enoe", full.names = TRUE)
archivos_sdemt <- lista_archivos[str_detect(lista_archivos, regex("sdemt", ignore_case = TRUE))]
archivos_coe1t <- lista_archivos[str_detect(lista_archivos, regex("coe1t", ignore_case = TRUE))]

In [21]:
# Crear un data frame con los nombres de archivos y sus años
df_archivos <- tibble(
  sdemt = archivos_sdemt,
  year = map_int(archivos_sdemt, extraer_anio)
) %>%
  left_join(
    tibble(
      coe1t = archivos_coe1t,
      year = map_int(archivos_coe1t, extraer_anio)
    ),
    by = "year"
  ) %>%
  filter(!is.na(sdemt) & !is.na(coe1t))%>%
        arrange(year)
df_archivos

sdemt,year,coe1t
<chr>,<int>,<chr>
datos/enoe/SDEMT407.csv,2007,datos/enoe/COE1T407.csv
datos/enoe/SDEMT408.csv,2008,datos/enoe/COE1T408.csv
datos/enoe/SDEMT409.csv,2009,datos/enoe/COE1T409.csv
datos/enoe/SDEMT410.csv,2010,datos/enoe/COE1T410.csv
datos/enoe/SDEMT411.csv,2011,datos/enoe/COE1T411.csv
datos/enoe/sdemt412.csv,2012,datos/enoe/coe1t412.csv
datos/enoe/sdemt413.csv,2013,datos/enoe/coe1t413.csv
datos/enoe/SDEMT414.csv,2014,datos/enoe/COE1T414.csv
datos/enoe/SDEMT415.csv,2015,datos/enoe/COE1T415.csv


In [22]:
#Estima empleo cultural de la Ciudad de México con metodología de cultura de la Ciudad de México
print("Población ocupada en cultura en la Ciudad de México de acuerdo a metodología de cultura cdmx")
empleo_cultural_cdmx <- df_archivos %>%
        select(-year) %>%
  mutate(data = map2(sdemt, coe1t, ~ estima_empleo_cultura(.x, .y))) %>%
  unnest(data)%>%
        select(year,ocupados,ocupados_cv,porcentaje,porcentaje_cv)%>%
        mutate(porcentaje=round(porcentaje,2))
empleo_cultural_cdmx

[1] "Población ocupada en cultura en la Ciudad de México de acuerdo a metodología de cultura cdmx"


Warning message:
"There were 23 warnings in `mutate()`.
The first warning was:
ℹ In argument: `data = map2(sdemt, coe1t, ~estima_empleo_cultura(.x, .y))`.
Caused by warning:
! One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)
ℹ Run `dplyr::last_dplyr_warnings()` to see the 22 remaining warnings."


year,ocupados,ocupados_cv,porcentaje,porcentaje_cv
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2012,138025,0.10084548,40.37,0.07194972
2013,121670,0.09421975,36.21,0.07388593
2014,100637,0.11085896,36.76,0.08324116
2015,102168,0.10818801,33.76,0.08506522
2016,116814,0.09992646,36.25,0.07802852
2017,138220,0.09385297,41.96,0.07122918
2018,136739,0.10213035,39.13,0.07603847
2019,135904,0.09715456,36.29,0.07528678
2020,109962,0.15542353,38.25,0.11466316


In [23]:
## Personas titulares en la Administración pública de la Ciudad de México

In [24]:
url<-"https://www.inegi.org.mx/contenidos/programas/cnge/"
segunda<-"/datosabiertos/m1/rec_huma_cnge"
fin<-"_csv.zip"

years<-c(2021:2024)

In [25]:
#Crea urls, descarga en carpeta "datos", descomprime y elimina archivos zip
for (i in years){
  url_descarga<-paste0(url,i,segunda,i,fin)
  temp <- tempfile()
  download.file(url_descarga, temp, mode = "wb")
  unzip(temp, exdir = "datos")
  unlink(temp)
}

In [26]:
##Función de titulares mujeres

titulares_mujeres <- function(year) {
  # Seleccionar archivo según el año
  archivo <- switch(as.character(year),
                    "2024" = "m1s1p3_cnge2024.csv",
                    "2021" = "m1s1p3_cnge2021.csv",
                    "2023" = "m1s1p2_cnge2023.csv",
                    "2022" = "m1s1p7_cnge2022.csv",
                    stop("Año no válido. Prueba con 2021, 2022, 2023 o 2024.")
  )

  # Leer y procesar los datos
df<-read_csv(paste0("datos/conjunto_de_datos/", archivo)) %>%
janitor::clean_names()
#Renomnbrar columnas si existen
   if ("sexo_e" %in% names(df)) {
    df <- df %>% rename(sexo_i = sexo_e)
  } else if ("sexo_f" %in% names(df)) {
    df <- df %>% rename(sexo_i = sexo_f)
  }

df%>%
    janitor::clean_names() %>%
    filter(entidad_a == 9) %>%
    group_by(sexo_i) %>%
    summarise(personas = n(), .groups = "drop") %>%
    mutate(porcentaje = personas / sum(personas) * 100) %>%
    filter(sexo_i == 2)%>%
mutate(year = year)

}


In [27]:
#Titulares mujeres

titulares_mujeres<-map_dfr(years,titulares_mujeres)
titulares_mujeres

Rows: 2271 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): nombre1, edades, totalca1, antigu_a, antigu_b
dbl (11): entidad_a, numeral, sexo_e, gdoestud_d, estagrad_b, emplante_a, pu...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2239 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): enticve1, nombre1, edades, totalca1, antigu_a, antigu_b
dbl (11): entidad_a, numeral, sexo_f, gdoestud_h, estagrad_c, emplante_e, pu...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2270 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (7): enticve1, nombre1, edades, totalca1, anti

sexo_i,personas,porcentaje,year
<dbl>,<int>,<dbl>,<int>
2,36,46.15385,2021
2,28,37.33333,2022
2,32,42.10526,2023
2,29,37.66234,2024


### Cuidados

In [30]:
url_cuenta_sat<-"https://www.inegi.org.mx/contenidos/programas/tnrh/2018/datosabiertos/tnrh_anual_csv.zip"

In [31]:
#Directorio cuidados
if (!dir.exists("datos/cuidados")) {
  dir.create("datos/cuidados")
}

In [32]:
#Descargar, descomprimir y borrar zip en la carpeta "datos"
temp <- tempfile()
download.file(url_cuenta_sat, temp, mode = "wb")
unzip(temp, exdir = "datos/cuidados")
unlink(temp)

In [199]:
###Leer tabla de tiempo destinado a cuidados


# Read the data and process it
tiempo_dest <- openxlsx::read.xlsx("https://www.inegi.org.mx/contenidos/programas/tnrh/2018/tabulados/ori/CSTNRHM_67.xlsx") %>%
  as_tibble() %>%
  slice(5:n()) %>%  # Skip first 4 rows
  {
    col_names <- as.character(.[1, ])  # Extract first row as column names

    # Replace NA or empty column names with unique placeholders
    col_names[is.na(col_names) | col_names == ""] <- paste0("Unnamed_", seq_along(col_names))

    # Ensure unique column names
    col_names <- make.names(col_names, unique = TRUE)

    colnames(.) <- col_names  # Assign new column names
    slice(., -1)  # Remove first row after setting it as header
  }

tiempo_dest<-tiempo_dest%>%
janitor::clean_names()%>%
#Seleccionar primera columna y todas aquellas que contengan "horas_semanales"
select(1,contains("horas_semanales_"))%>%rename(cuidados=1)%>%
#Eliminar espacios atrás en primera columna
mutate(cuidados=str_trim(cuidados, side = "left"))%>%
#Filtrar si la primera columna contiene "Total"
filter(str_detect(cuidados,"Total"))%>%
#Dejar fila 10
slice(10)%>%
#Quitar columnas que contengan "horas_semanales_per_capita"
select(-contains("horas_semanales_per_capita"))%>%
#Pivot long
pivot_longer(-c(cuidados),names_to="sexo",values_to="horas_semanales")%>%
filter(as.numeric(gsub("horas_semanales_", "", sexo)) %% 3 != 0)%>%
select(!sexo)%>%
#Crear variable sexo donde primero sea mujer y después hombre
mutate(sexo=ifelse(row_number()%%2==0,"Hombre","Mujer"))%>%
  mutate(year = 2019 + ((row_number() - 1) %/% 2))%>%
  mutate(horas_semanales=as.numeric(horas_semanales))

tiempo_dest<-tiempo_dest%>%
pivot_wider(names_from = sexo, values_from = horas_semanales)%>%
#Relación con dos decimales
mutate(rel_muj_hom=round(Mujer/Hombre,2))


tiempo_dest



Warning message in col_names[is.na(col_names) | col_names == ""] <- paste0("Unnamed_", :
"number of items to replace is not a multiple of replacement length"


cuidados,year,Mujer,Hombre,rel_muj_hom
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Total,2019,142688521,62813404,2.27
Total,2020,145605145,65376670,2.23
Total,2021,148272245,68606916,2.16
Total,2022,153227469,72853711,2.10
Total,2023,157280688,76898733,2.05
